In [1]:
# Random Forest : RF

# RF-Regressor

# carsdekho.com dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv('cardekho.csv')

df.drop(['Unnamed: 0','car_name','brand'],axis=1,inplace=True)

df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df.isnull().sum()

model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model              15411 non-null  object 
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
 10  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1.3+ MB


In [12]:
# separating numeric features -(discrete , continuous) , categorical features 

num_cols = df.select_dtypes(exclude='object').columns
cat_cols = df.select_dtypes(include='object').columns

dis_feats = [feat for feat in num_cols if (len(df[feat].unique()))>=25]
cont_feats = [feat for feat in num_cols if feat not in dis_feats]

In [13]:
# labesls & target
X = df.drop('selling_price',axis=1)
y = df['selling_price']

In [20]:
# Encodind : Label & OneHot

len(df['model'].unique())

120

In [22]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

oh_cols = ['seller_type','fuel_type','transmission_type']

oh = OneHotEncoder()

preprocessor = ColumnTransformer(
    [('OneHotEn',oh,oh_cols)],
    remainder = 'passthrough'
)

X = preprocessor.fit_transform(X)

In [31]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [32]:
# model fitting & training

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [33]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

def evaluate_models(true_vals,pred_vals):
    mae = mean_absolute_error(true_vals,pred_vals)
    mse = mean_squared_error(true_vals,pred_vals)
    rmse = np.sqrt(mse)
    r2 = r2_score(true_vals,pred_vals)

    return mae,rmse,r2

In [35]:
models = {
    "Linear Regression":LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'KNN-Reg':KNeighborsRegressor(),
    'DT-Reg' :DecisionTreeRegressor(),
    'RF-Reg' :RandomForestRegressor()
}

for name,model in models.items():
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_models(y_train,y_train_pred)
    model_test_mae , model_test_rmse, model_test_r2 = evaluate_models(y_test,y_test_pred)

    print(f'------model performance of {name} for training set-------')
    print(f' mae = {model_train_mae:.3f}')
    print(f' rmse = {model_train_rmse:.3f}')
    print(f' r2_score = {model_train_r2:.3f}')
    print('-'*15)
    print(f'------model performance of {name} for test set-------')
    print(f' mae = {model_test_mae:.3f}')
    print(f' rmse = {model_test_rmse:.3f}')
    print(f' r2_score = {model_test_r2:.3f}')

------model performance of Linear Regression for training set-------
 mae = 268101.607
 rmse = 553855.667
 r2_score = 0.622
---------------
------model performance of Linear Regression for test set-------
 mae = 279618.579
 rmse = 502543.593
 r2_score = 0.665
------model performance of Lasso for training set-------
 mae = 268099.903
 rmse = 553855.671
 r2_score = 0.622
---------------
------model performance of Lasso for test set-------
 mae = 279615.401
 rmse = 502543.012
 r2_score = 0.665
------model performance of Ridge for training set-------
 mae = 268078.925
 rmse = 553856.301
 r2_score = 0.622
---------------
------model performance of Ridge for test set-------
 mae = 279574.856
 rmse = 502538.500
 r2_score = 0.665
------model performance of KNN-Reg for training set-------
 mae = 193685.707
 rmse = 485514.714
 r2_score = 0.709
---------------
------model performance of KNN-Reg for test set-------
 mae = 255604.517
 rmse = 523547.121
 r2_score = 0.636
------model performance of D

In [42]:
# selecting best performing model & hyperparameter tunning them to get even better performance

dtr_params = {'max_depth':[1,2,5,10],'splitter':['best','random']}

rfr_params = {
    'max_depth':[5,8,15,None,10],
    'max_features':[5,7,'auto',8],
    'min_samples_split':[2,8,15,20],
    'n_estimators':[100,200,500,1000]
}

randomcv_models = [
    ('DF-Reg',DecisionTreeRegressor(),dtr_params),
    ('RF-Reg',RandomForestRegressor(),rfr_params)
]

In [50]:
# Hyperparam tunning

import warnings;warnings.filterwarnings('ignore')

from sklearn.model_selection import RandomizedSearchCV

model_params = {}

for name,model,params in randomcv_models:
    random = RandomizedSearchCV(model,
                                param_distributions=params,
                                cv=3)

    random.fit(X_train,y_train)

    model_params[name]=random.best_params_

for model_name in model_params:
    print(f'------ Best Params for {model_name}------')
    print(model_params[model_name])

------ Best Params for DF-Reg------
{'splitter': 'best', 'max_depth': 10}
------ Best Params for RF-Reg------
{'n_estimators': 200, 'min_samples_split': 8, 'max_features': 8, 'max_depth': None}


In [51]:
# substituting best params back to models

models = {
    'DT-Reg':DecisionTreeRegressor(splitter='best' ,max_depth =10),
    'RF-Reg':RandomForestRegressor(n_estimators=200, min_samples_split=8, max_features=8, max_depth=None)
}

for name,model in models.items():
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred=model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_models(y_train,y_train_pred)
    model_test_mae , model_test_rmse, model_test_r2 = evaluate_models(y_test,y_test_pred)

    print(f'------model performance of {name} for training set-------')
    print(f' mae = {model_train_mae:.3f}')
    print(f' rmse = {model_train_rmse:.3f}')
    print(f' r2_score = {model_train_r2:.3f}')
    print('-'*15)
    print(f'------model performance of {name} for test set-------')
    print(f' mae = {model_test_mae:.3f}')
    print(f' rmse = {model_test_rmse:.3f}')
    print(f' r2_score = {model_test_r2:.3f}')

------model performance of DT-Reg for training set-------
 mae = 87464.270
 rmse = 159038.308
 r2_score = 0.969
---------------
------model performance of DT-Reg for test set-------
 mae = 125858.610
 rmse = 317574.015
 r2_score = 0.866
------model performance of RF-Reg for training set-------
 mae = 66587.785
 rmse = 187436.765
 r2_score = 0.957
---------------
------model performance of RF-Reg for test set-------
 mae = 100513.850
 rmse = 222862.444
 r2_score = 0.934
